In [7]:
import sys
!"{sys.executable}" -m pip install pandas matplotlib numpy scikit-learn


  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ---------------------------------------- 0.0/114.6 kB ? eta -:--:--
     --- ------------------------------------ 10.2/114.6 kB ? eta -:--:--
     --------- --------------------------- 30.7/114.6 kB 660.6 kB/s eta 0:00:01
     -------------------------------------- 114.6/114.6 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   --- ------------------------------------ 0.6/8.1 MB 19.8 MB/s eta 0:00:01
   --------- ------------------------------ 2.0/8.1 MB 25.3 MB/s eta 0:00:01
   --------------- ------------------------ 3.2/8.1 MB 25.7 MB/s eta 0:00:01
   -------------------- ------------------- 4.1/8.1 MB 29.0 MB/s eta 0:00:01
   -------------------- ------------------- 4.1/8.1 MB 29.0 MB/s eta 0:00:01
   -------------------- ------------------- 4.1/8.1 MB 29.0 MB/s eta 0:00:01
   -------------------- ------------------- 4.1/8.1 MB 29.0 MB/s eta 0:00:01
   ------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\stanl\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
print(pd.__version__)


2.3.3


In [5]:
'''
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Load data
train = pd.read_csv("aluminum_coldRoll_train.csv")
test = pd.read_csv("aluminum_coldRoll_testNoY.csv")

# Separate features and target
X = train.drop(columns=["y_passXtremeDurability"])
y = train["y_passXtremeDurability"]

# Encode any categorical features (auto-detect object types)
for col in X.select_dtypes(include="object").columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    if col in test.columns:
        test[col] = le.transform(test[col])

# Prepare cross-validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_accuracies = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X, y), 1):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
    rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    rf.fit(X_train, y_train)
    
    val_preds = rf.predict(X_val)
    acc = accuracy_score(y_val, val_preds)
    fold_accuracies.append(acc)
    print(f"Fold {fold} Accuracy: {acc:.4f}")

print(f"\nAverage CV Accuracy: {np.mean(fold_accuracies):.4f}")

# Retrain on full training data
final_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
final_model.fit(X, y)

# Encode test features if needed
X_test = test.copy()
for col in X_test.select_dtypes(include="object").columns:
    le = LabelEncoder()
    X_test[col] = le.fit_transform(X_test[col])

# Predict probabilities on test data
test_probs = final_model.predict_proba(X_test)[:, 1]

# Format submission
submission = pd.DataFrame({
    "Id": test["Id"],
    "y_passXtremeDurability": test_probs
})

submission.to_csv("random_forest_predictions.csv", index=False)

'''

Fold 1 Accuracy: 0.7839
Fold 2 Accuracy: 0.7804
Fold 3 Accuracy: 0.7843
Fold 4 Accuracy: 0.7818
Fold 5 Accuracy: 0.7814

Average CV Accuracy: 0.7823


KeyError: 'Id'

In [3]:
'''
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier

# Load data
train = pd.read_csv("aluminum_coldRoll_train.csv")

X = train.drop(columns=["y_passXtremeDurability"])
y = train["y_passXtremeDurability"]

# Encode categorical variables
for col in X.select_dtypes(include="object").columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Range of k to try
k_values = range(50, 70, 4)
results = []

# 5-fold cross-validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for k in k_values:
    acc_scores = []

    for train_idx, val_idx in kf.split(X_scaled, y):
        X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)
        preds = knn.predict(X_val)

        acc = accuracy_score(y_val, preds)
        acc_scores.append(acc)

    avg_acc = np.mean(acc_scores)
    results.append((k, avg_acc))
    print(f"k = {k}, Average CV Accuracy = {avg_acc:.4f}")

# Find best k
best_k, best_score = max(results, key=lambda x: x[1])
print(f"\n✅ Best k: {best_k} with CV Accuracy = {best_score:.4f}")
'''

k = 50, Average CV Accuracy = 0.7700
k = 54, Average CV Accuracy = 0.7697
k = 58, Average CV Accuracy = 0.7699
k = 62, Average CV Accuracy = 0.7697
k = 66, Average CV Accuracy = 0.7701

✅ Best k: 66 with CV Accuracy = 0.7701


In [4]:
'''
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load data
train = pd.read_csv("aluminum_coldRoll_train.csv")
test = pd.read_csv("aluminum_coldRoll_testNoY.csv")

# Separate features and target
X = train.drop(columns=["y_passXtremeDurability"])
y = train["y_passXtremeDurability"]

# Encode categorical variables
for col in X.select_dtypes(include="object").columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    if col in test.columns:
        test[col] = le.transform(test[col])

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
test_scaled = scaler.transform(test)

# Cross-validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled, y), 1):
    X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    lr = LogisticRegression(max_iter=1000)
    lr.fit(X_train, y_train)
    preds = lr.predict(X_val)
    acc = accuracy_score(y_val, preds)
    cv_scores.append(acc)
    print(f"Fold {fold} Accuracy: {acc:.4f}")

print(f"\nAverage CV Accuracy: {np.mean(cv_scores):.4f}")

# Retrain on full data
final_model = LogisticRegression(max_iter=1000)
final_model.fit(X_scaled, y)

# Predict on test set
test_probs = final_model.predict_proba(test_scaled)[:, 1]

# Output submission CSV
submission = pd.DataFrame({
    "Id": test["Id"],
    "y_passXtremeDurability": test_probs
})
submission.to_csv("logistic_regression_predictions.csv", index=False)
'''

Fold 1 Accuracy: 0.7155
Fold 2 Accuracy: 0.7099
Fold 3 Accuracy: 0.7094
Fold 4 Accuracy: 0.7109
Fold 5 Accuracy: 0.7127

Average CV Accuracy: 0.7117


KeyError: 'Id'

In [2]:
'''
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load data
train = pd.read_csv("aluminum_coldRoll_train.csv")
test = pd.read_csv("aluminum_coldRoll_testNoY.csv")

# Separate features and target
X = train.drop(columns=["y_passXtremeDurability"])
y = train["y_passXtremeDurability"]

# Encode categorical features
for col in X.select_dtypes(include="object").columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    if col in test.columns:
        test[col] = le.transform(test[col])

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
test_scaled = scaler.transform(test)

# 5-Fold CV
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled, y), 1):
    X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model = LogisticRegression(penalty='l1', solver='liblinear', max_iter=1000)
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    acc = accuracy_score(y_val, preds)
    cv_scores.append(acc)
    print(f"Fold {fold} Accuracy: {acc:.4f}")

print(f"\nAverage CV Accuracy: {np.mean(cv_scores):.4f}")

# Retrain on full data
final_model = LogisticRegression(penalty='l1', solver='liblinear', max_iter=1000)
final_model.fit(X_scaled, y)

# Predict probabilities on test set
test_probs = final_model.predict_proba(test_scaled)[:, 1]

# Submission
submission = pd.DataFrame({
    "Id": test["Id"],
    "y_passXtremeDurability": test_probs
})
submission.to_csv("lasso_logistic_predictions.csv", index=False)
'''

Fold 1 Accuracy: 0.7156
Fold 2 Accuracy: 0.7099
Fold 3 Accuracy: 0.7094
Fold 4 Accuracy: 0.7110
Fold 5 Accuracy: 0.7127

Average CV Accuracy: 0.7117


KeyError: 'Id'

In [3]:
'''
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load data
train = pd.read_csv("aluminum_coldRoll_train.csv")
test = pd.read_csv("aluminum_coldRoll_testNoY.csv")

# Separate features and target
X = train.drop(columns=["y_passXtremeDurability"])
y = train["y_passXtremeDurability"]

# Encode categorical variables
for col in X.select_dtypes(include="object").columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    if col in test.columns:
        test[col] = le.transform(test[col])

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
test_scaled = scaler.transform(test)

# Cross-validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled, y), 1):
    X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    ridge_model = LogisticRegression(penalty='l2', solver='liblinear', max_iter=1000)
    ridge_model.fit(X_train, y_train)
    preds = ridge_model.predict(X_val)
    acc = accuracy_score(y_val, preds)
    cv_scores.append(acc)
    print(f"Fold {fold} Accuracy: {acc:.4f}")

print(f"\nAverage CV Accuracy: {np.mean(cv_scores):.4f}")

# Retrain on full training data
final_model = LogisticRegression(penalty='l2', solver='liblinear', max_iter=1000)
final_model.fit(X_scaled, y)

# Predict probabilities on test set
test_probs = final_model.predict_proba(test_scaled)[:, 1]

# Save to submission CSV
submission = pd.DataFrame({
    "Id": test["Id"],
    "y_passXtremeDurability": test_probs
})
submission.to_csv("ridge_logistic_predictions.csv", index=False)
'''

Fold 1 Accuracy: 0.7156
Fold 2 Accuracy: 0.7098
Fold 3 Accuracy: 0.7093
Fold 4 Accuracy: 0.7109
Fold 5 Accuracy: 0.7127

Average CV Accuracy: 0.7117


KeyError: 'Id'

In [2]:
# LASSO REGRESSION
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load data
train = pd.read_csv("aluminum_coldRoll_train.csv")

# Separate features and target
X = train.drop(columns=["y_passXtremeDurability"])
y = train["y_passXtremeDurability"]

# Encode categorical variables
for col in X.select_dtypes(include="object").columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 5-fold cross-validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
mse_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled, y), 1):
    X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model = LogisticRegression(penalty='l1', solver='liblinear', max_iter=1000)
    model.fit(X_train, y_train)
    preds_proba = model.predict_proba(X_val)[:, 1]
    mse = mean_squared_error(y_val, preds_proba)
    mse_scores.append(mse)
    print(f"Fold {fold} MSE: {mse:.4f}")

print(f"\nAverage MSE: {np.mean(mse_scores):.4f}")


Fold 1 MSE: 0.1890
Fold 2 MSE: 0.1905
Fold 3 MSE: 0.1903
Fold 4 MSE: 0.1913
Fold 5 MSE: 0.1898

Average MSE: 0.1902


In [3]:
# RIDGE REGRESSION
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load training data
train = pd.read_csv("aluminum_coldRoll_train.csv")

# Separate features and target
X = train.drop(columns=["y_passXtremeDurability"])
y = train["y_passXtremeDurability"]

# Encode categorical variables
for col in X.select_dtypes(include="object").columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 5-Fold CV with Ridge Logistic Regression
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
mse_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled, y), 1):
    X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model = LogisticRegression(penalty='l2', solver='liblinear', max_iter=1000)
    model.fit(X_train, y_train)
    preds_proba = model.predict_proba(X_val)[:, 1]
    mse = mean_squared_error(y_val, preds_proba)
    mse_scores.append(mse)
    print(f"Fold {fold} MSE: {mse:.4f}")

print(f"\nAverage MSE: {np.mean(mse_scores):.4f}")


Fold 1 MSE: 0.1890
Fold 2 MSE: 0.1905
Fold 3 MSE: 0.1903
Fold 4 MSE: 0.1913
Fold 5 MSE: 0.1898

Average MSE: 0.1902


In [5]:
# RANDOM FOREST
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load training data
train = pd.read_csv("aluminum_coldRoll_train.csv")

# Separate features and target
X = train.drop(columns=["y_passXtremeDurability"])
y = train["y_passXtremeDurability"]

# Encode categorical variables
for col in X.select_dtypes(include="object").columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 5-Fold CV with Random Forest (faster settings)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
mse_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled, y), 1):
    X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model = RandomForestClassifier(
        n_estimators=50,          # reduced number of trees
        max_depth=10,             # limit tree depth
        random_state=42,
        n_jobs=-1                 # use all CPU cores
    )
    model.fit(X_train, y_train)
    preds_proba = model.predict_proba(X_val)[:, 1]
    mse = mean_squared_error(y_val, preds_proba)
    mse_scores.append(mse)
    print(f"Fold {fold} MSE: {mse:.4f}")

print(f"\nAverage MSE: {np.mean(mse_scores):.4f}")


Fold 1 MSE: 0.1533
Fold 2 MSE: 0.1546
Fold 3 MSE: 0.1534
Fold 4 MSE: 0.1554
Fold 5 MSE: 0.1544

Average MSE: 0.1542


In [ ]:
# K-NEAREST NEIGHBORS, use K = 39
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, StandardScaler
import matplotlib.pyplot as plt

# Load data
train = pd.read_csv("aluminum_coldRoll_train.csv")
X = train.drop(columns=["y_passXtremeDurability"])
y = train["y_passXtremeDurability"]

# Encode categoricals
for col in X.select_dtypes(include="object").columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Evaluate selected k values
k_values = [13, 35, 39, 43, 47, 51, 55]
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
mse_scores = []

for k in k_values:
    fold_mse = []
    for train_idx, val_idx in kf.split(X_scaled, y):
        X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model = KNeighborsClassifier(n_neighbors=k)
        model.fit(X_train, y_train)
        preds_proba = model.predict_proba(X_val)[:, 1]
        mse = mean_squared_error(y_val, preds_proba)
        fold_mse.append(mse)

    avg_mse = np.mean(fold_mse)
    mse_scores.append(avg_mse)
    print(f"k = {k}, Average MSE: {avg_mse:.4f}")

# Plot MSE vs k
plt.plot(k_values, mse_scores, marker='o')
plt.xlabel("k")
plt.ylabel("Average MSE")
plt.title("5-Fold CV MSE for Different k (KNN)")
plt.grid(True)
plt.show()

# Best k
best_k = k_values[np.argmin(mse_scores)]
print(f"\nBest k: {best_k} with MSE = {min(mse_scores):.4f}")


k = 13, Average MSE: 0.1622
k = 35, Average MSE: 0.1578
k = 39, Average MSE: 0.1577
k = 43, Average MSE: 0.1578
k = 47, Average MSE: 0.1578
k = 51, Average MSE: 0.1579


In [ ]:
import pandas as pd
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load training data
train = pd.read_csv("aluminum_coldRoll_train.csv")

# Separate features and target
X = train.drop(columns=["y_passXtremeDurability"])
y = train["y_passXtremeDurability"]

# Encode categorical variables
for col in X.select_dtypes(include="object").columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 5-Fold Cross-Validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
mse_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled, y), 1):
    X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model = LinearDiscriminantAnalysis()
    model.fit(X_train, y_train)
    preds_proba = model.predict_proba(X_val)[:, 1]
    mse = mean_squared_error(y_val, preds_proba)
    mse_scores.append(mse)
    print(f"Fold {fold} MSE: {mse:.4f}")

print(f"\nAverage MSE: {np.mean(mse_scores):.4f}")
